In [53]:
import polars as pl
import duckdb
from pathlib import Path

# Handle paths

In [54]:
staging_tool_path = Path().absolute().parent.parent / "data" / "datalake" / "staging" / "vr_data"

vr_full_load_jsons = list(staging_tool_path.glob("**/*.json"))
print(len(list(vr_full_load_jsons)))

1


# Visualization on one loaded json file

In [55]:
polars_df = pl.read_json(vr_full_load_jsons[0])
polars_df.head(2)


trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
i64,str,i64,str,str,str,str,bool,bool,i64,str,str,list[struct[16]]
1,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,286822491217,"""REGULAR""","""2023-07-05T11:…","[{""HKI"",1,""FI"",""DEPARTURE"",true,""10"",false,""2023-11-01T04:55:00.000Z"",null,null,0,[],1,true,""2023-11-01T04:55:21.000Z"",{""KUPLA"",true,""2023-11-01T04:53:40.000Z""}}, {""PSL"",10,""FI"",""ARRIVAL"",true,""4"",false,""2023-11-01T05:00:00.000Z"",null,null,0,[],1,true,""2023-11-01T05:00:14.000Z"",{null,null,null}}, … {""JNS"",460,""FI"",""ARRIVAL"",true,""1"",false,""2023-11-01T09:41:00.000Z"",""2023-11-01T09:39:41.000Z"",""LIIKE_AUTOMATIC"",-1,[],1,true,""2023-11-01T09:39:41.000Z"",{null,null,null}}]"
2,"""2023-11-01""",10,"""vr""","""S""","""Long-distance""","""""",false,false,286821604082,"""REGULAR""","""2023-07-05T11:…","[{""JNS"",460,""FI"",""DEPARTURE"",true,""1"",false,""2023-11-01T02:58:00.000Z"",null,null,0,[],2,true,""2023-11-01T02:58:00.000Z"",{""KUPLA"",true,""2023-11-01T02:53:10.000Z""}}, {""PLT"",1070,""FI"",""ARRIVAL"",false,"""",false,""2023-11-01T02:58:42.000Z"",""2023-11-01T02:58:42.000Z"",""LIIKE_AUTOMATIC"",0,[],2,null,null,{null,null,null}}, … {""HKI"",1,""FI"",""ARRIVAL"",true,""9"",false,""2023-11-01T07:30:00.000Z"",null,null,1,[],2,true,""2023-11-01T07:31:07.000Z"",{null,null,null}}]"


In [56]:
SCHEMA = {
    "trainNumber": pl.Int64,
    "departureDate": pl.Utf8,
    "operatorUICCode": pl.Int64,
    "operatorShortCode": pl.Utf8,
    "trainType": pl.Utf8,
	"trainCategory": pl.Utf8,
	"commuterLineID": pl.Utf8,
    "runningCurrently": pl.Boolean,
	"cancelled": pl.Boolean,
    "deleted": pl.Boolean,
    "version": pl.Int64,
    "timetableType": pl.Utf8,
	"timetableAcceptanceDate": pl.Utf8,
    "timeTableRows": pl.List(
        pl.Struct({
            "stationShortCode": pl.Utf8,
            "stationUICCode": pl.Int64,
            "countryCode": pl.Utf8,
            "type": pl.Utf8,
            "trainStopping": pl.Boolean,
            "commercialStop": pl.Boolean,
            "commercialTrack": pl.Utf8,
            "cancelled": pl.Boolean,
            "scheduledTime": pl.Utf8,
            "liveEstimateTime": pl.Utf8,
            "estimateSource": pl.Utf8,
            "unknownDelay": pl.Boolean,
            "actualTime": pl.Utf8,
            "differenceInMinutes": pl.Int64,
            "causes": pl.List(
                pl.Struct(
                    {
                        "passengerTerm": pl.Utf8,
                        "categoryCode": pl.Utf8,
                        "categoryName": pl.Utf8,
                        "validFrom": pl.Utf8,
                        "validTo": pl.Utf8,
                        "id": pl.Int32,
                        "detailedCategoryCode": pl.Utf8,
                        "detailedCategoryName": pl.Utf8,
                        "thirdCategoryCode": pl.Utf8,
                        "thirdCategoryName": pl.Utf8,
                        "description": pl.Utf8,
                        "categoryCodeId": pl.Int32,
                        "detailedCategoryCodeId": pl.Int32,
                        "thirdCategoryCodeId": pl.Int32,
                    }
                )
            ),
            "trainReady": pl.List(
                pl.Struct(
                    {
                        "source": pl.Utf8,
                        "accepted": pl.Boolean,
                        "timestamp": pl.Utf8,   
                    }
                ),
            ),
        }),
    )
}

#polars_df = pl.read_json(vr_full_load_jsons[0], schema=SCHEMA)
#polars_df

In [57]:
polars_df = pl.read_json(vr_full_load_jsons[0], schema=SCHEMA)
polars_df.head(1)

trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,deleted,version,timetableType,timetableAcceptanceDate,timeTableRows
i64,str,i64,str,str,str,str,bool,bool,bool,i64,str,str,list[struct[16]]
1,"""2023-11-01""",10,"""vr""","""IC""","""Long-distance""","""""",false,false,null,286822491217,"""REGULAR""","""2023-07-05T11:…","[{""HKI"",1,""FI"",""DEPARTURE"",true,true,""10"",false,""2023-11-01T04:55:00.000Z"",null,null,null,""2023-11-01T04:55:21.000Z"",0,[],null}, {""PSL"",10,""FI"",""ARRIVAL"",true,true,""4"",false,""2023-11-01T05:00:00.000Z"",null,null,null,""2023-11-01T05:00:14.000Z"",0,[],null}, … {""JNS"",460,""FI"",""ARRIVAL"",true,true,""1"",false,""2023-11-01T09:41:00.000Z"",""2023-11-01T09:39:41.000Z"",""LIIKE_AUTOMATIC"",null,""2023-11-01T09:39:41.000Z"",-1,[],null}]"


# Lazy df

In [58]:
lazy_df = pl.scan_ndjson(vr_full_load_jsons[0], schema=SCHEMA)
#print(lazy_df.limit(1))
# lazy_df.limit(3).collect()

# Write to Warehouse (DuckDB)

In [59]:
duckdb_path = Path().absolute().parent.parent / "data" / "warehouse" / "vr.duckdb"
duckdb_path.is_file()
conn = duckdb.connect(database=str(duckdb_path), read_only=False)

In [62]:
#conn.sql("SHOW")
conn.sql("SELECT * FROM lazy_df").pl()

trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,deleted,version,timetableType,timetableAcceptanceDate,timeTableRows
i64,str,i64,str,str,str,str,bool,bool,bool,i64,str,str,list[i32]
null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [64]:
conn.sql("""
CREATE SCHEMA IF NOT EXISTS medallion_bronze;     
CREATE OR REPLACE TABLE vr.medallion_bronze.method_b_traintest AS (  
    WITH traintest_data AS (
        SELECT 
            md5(
                trainNumber || departureDate || operatorUICCode 
            ) as route_sk, --Surrogate Key
            * 
        FROM lazy_df
    )
            
    SELECT * FROM traintest_data
);
""")

conn.sql("""SELECT * FROM medallion_bronze.method_b_traintest LIMIT 1""")

┌──────────┬─────────────┬───────────────┬───┬─────────┬───────────────┬──────────────────────┬───────────────┐
│ route_sk │ trainNumber │ departureDate │ … │ version │ timetableType │ timetableAcceptanc…  │ timeTableRows │
│ varchar  │    int64    │    varchar    │   │  int64  │    varchar    │       varchar        │    int32[]    │
├──────────┼─────────────┼───────────────┼───┼─────────┼───────────────┼──────────────────────┼───────────────┤
│ NULL     │        NULL │ NULL          │ … │    NULL │ NULL          │ NULL                 │ NULL          │
├──────────┴─────────────┴───────────────┴───┴─────────┴───────────────┴──────────────────────┴───────────────┤
│ 1 rows                                                                                 15 columns (7 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [46]:
conn.close()